In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [2]:
# examine the first 5 rows of X (including the feature names)
import pandas as pd
df = pd.read_excel("data.xls")
li = df[df.isnull().T.any()].index
for i in li:
    df.drop(index=i,axis = 1,inplace = True)
df[df.isnull().T.any()]
df

,tweet,polarity,subjectivity,vader_neg,vader_neu,vader_pos,vader_compound
0,Same folks said daikon paste could treat a cyt...,0.000000,0.125000,0.000,0.787,0.213,0.4019
1,While the world has been on the wrong side of ...,-0.500000,0.900000,0.125,0.766,0.109,-0.1027
2,coronavirus SputnikV AstraZeneca PfizerBioNTec...,0.000000,0.033333,0.000,0.882,0.118,0.2500
3,Facts are immutable Senator even when you re n...,-0.050000,0.550000,0.000,1.000,0.000,0.0000
4,Explain to me again why we need a vaccine wher...,0.000000,0.000000,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...
7656,21 hours status post PfizerBiontech vaccine My...,-0.155556,0.288889,0.191,0.671,0.138,-0.3291
7657,What a week 2nd dose complete fantastically sm...,0.350000,0.450000,0.122,0.878,0.000,-0.3612
7658,Please Retweet to encourage others Got my 2nd ...,0.000000,0.000000,0.000,0.570,0.430,0.8074
7659,Grateful to be fully vaccinated but these side...,0.000000,0.000000,0.257,0.644,0.099,-0.6369


In [3]:
import numpy as np
df1 = pd.read_csv('Finished_vaccination_tweets_手动标记.csv',encoding='ISO-8859-1')
df1 = df1[(df1['code-attitude'] == '0') | (df1['code-attitude'] == '1') | (df1['code-attitude'] == '-1')]
df1['text']

2       Facts are immutable, Senator, even when you're...
3       Explain to me again why we need a vaccine @Bor...
4       Does anyone have any useful advice/guidance fo...
5       it is a bit sad to claim the fame for success ...
6       There have not been many bright days in 2020 b...
                              ...                        
3101    #PublicHealth #COVID19 #Modernavaccine #Modern...
3102    @crashoverrideee #COVID19 Vaccine Update for #...
3103    Dr Fun's I Feel Good - My effort at a PSA, ple...
3104    @Writer_DG I got dose #2 Friday. Have to admit...
3105    Feeling very privileged to have had my first c...
Name: text, Length: 247, dtype: object

In [4]:
df['polarity_new']=5
df.loc[df['polarity']==0,['polarity_new']]=0
df.loc[df['polarity']>0,['polarity_new']]=1
df.loc[df['polarity']<0,['polarity_new']]=-1
df['polarity_new'].value_counts()

 1    3461
 0    3369
-1     827
Name: polarity_new, dtype: int64

In [5]:
X = df['tweet']
Y = df['polarity_new']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=123)
X_train.shape

(6125,)

In [7]:
num_features = 200
vect = CountVectorizer(max_features=num_features)
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)
X_test_dtm

<1532x200 sparse matrix of type '<class 'numpy.int64'>'
	with 13212 stored elements in Compressed Sparse Row format>

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
# instantiate a logistic regression model
logreg = LogisticRegression(multi_class='multinomial', solver="lbfgs", C=10, random_state=123)

In [9]:
%time logreg.fit(X_train_dtm,y_train)

CPU times: user 235 ms, sys: 10.3 ms, total: 245 ms
Wall time: 258 ms


/Users/linqiaoyi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=10, multi_class='multinomial', random_state=123)

In [10]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def custom_preprocessor(text):
    '''
    Make text lowercase, remove text in square brackets,remove links,remove special characters
    and remove words containing numbers.
    '''
    text = text.lower()
    text = re.sub('@[^\s]+','',text)
    text = re.sub(r'\B#\S+','',text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) # remove special chars
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    words = word_tokenize(text)
    words = [i for i in words if i not in stopwords.words()]
    text = ' '.join(words)
    
    return text

In [12]:
df1['text'] = pd.DataFrame(df1['text'].astype(str))
df1['text_clean']=df1['text'].apply(custom_preprocessor)
t = vect.transform(df1['text_clean'])
y_test_pred = logreg.predict(t)
df1['code-attitude'] = pd.DataFrame(df1['code-attitude'].astype(int))
from sklearn.metrics import accuracy_score, precision_score, recall_score
print("Accuracy:{}".format(accuracy_score(df1['code-attitude'], y_test_pred)))
print("Precision:{}".format(precision_score(df1['code-attitude'], y_test_pred,average='micro')))
print("Recall:{}".format(recall_score(df1['code-attitude'], y_test_pred,average='micro')))

Accuracy:0.4898785425101215
Precision:0.4898785425101215
Recall:0.4898785425101215


In [13]:
df['tweet_clean']=df['tweet'].apply(custom_preprocessor)
X = df['tweet_clean']
Y = df['polarity_new']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=123)
X_train

1766    video dubai residents queue receive vaccinatio...
7267    better half supposed get first pfizerbiontech ...
1430    neither pfizerbiontech produced experimental m...
3551    uk apart oxfordvaccine moderna pfizerbiontech ...
2405                     counted vaccinated first jab sec
                              ...                        
4063    bmj patients died norway shortly receiving pfi...
1346    joe biden receives first dose pfizer covid vac...
3457    laboratory tests shown pfizerbiontech vaccine ...
7537    hubby fully vaccinated thank science pfizerbio...
3585    anyone ireland anywhere else thinking receivin...
Name: tweet_clean, Length: 6125, dtype: object

In [14]:
vect = CountVectorizer(ngram_range=(1,2))
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)
X_train_dtm

<6125x39898 sparse matrix of type '<class 'numpy.int64'>'
	with 100109 stored elements in Compressed Sparse Row format>

In [15]:
%%time

logreg = LogisticRegression()
logreg.fit(X_train_dtm,y_train)


CPU times: user 3.25 s, sys: 66.9 ms, total: 3.32 s
Wall time: 1.8 s


/Users/linqiaoyi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [18]:
y_test_pred = logreg.predict(X_test_dtm)
print("Accuracy:{}".format(accuracy_score(y_test, y_test_pred)))
print("Precision:{}".format(precision_score(y_test, y_test_pred,average='micro')))
print("Recall:{}".format(recall_score(y_test, y_test_pred,average='micro')))

Accuracy:0.8178851174934726
Precision:0.8178851174934726
Recall:0.8178851174934726


In [19]:
df1['text'] = pd.DataFrame(df1['text'].astype(str))
df1['text_clean']=df1['text'].apply(custom_preprocessor)
t = vect.transform(df1['text_clean'])
y_test_pred = logreg.predict(t)
df1['code-attitude'] = pd.DataFrame(df1['code-attitude'].astype(int))
from sklearn.metrics import accuracy_score, precision_score, recall_score
print("Accuracy:{}".format(accuracy_score(df1['code-attitude'], y_test_pred)))


Accuracy:0.5465587044534413
